- https://ineuron.ai/sitemap.xml

In [1]:
!pip install langchain
!pip install bitsandbytes accelerate transformers -q
!pip install sentence_transformers
!pip install unstructured
!pip install -U langchain-community
!pip install pinecone-client==2.2.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
import pinecone, transformers, torch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, AutoModelForCausalLM


/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### `Extract Data from URL`

In [54]:
urls = ['https://ineuron.ai']

In [55]:
loader = UnstructuredURLLoader(urls = urls)
data = loader.load()
data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


[Document(page_content='Learning with iNeuron made \xa0<>\n\nTake your career to the next level with industry ready programs,\n\nAn entire learning ecosystem at your fingertips to make learning fun.\n\nChoose from a range of tech programs and make your next big career switch.\n\nExplore Courses\n\n55%\n\nAverage Salary Hike\n\n400+\n\nDifferent Courses\n\n10000+\n\nCareer Transitions\n\n400+\n\nHiring Partners\n\nLIVE NOW\n\nSupport System\n\nOur support system is live again, this time it is bigger, better and faster.\n\nExperience a tech community like never seen before\n\nTake me there\n\nOur Courses\n\nView all\n\nView all\n\nSuccess Stories\n\nView all\n\nFresher\n\nAbhisekh Bhuyan\n\nMLOps engineer\n\nI got job as an MLOps engineer at synapsica at 13 LPA PPO because of "End to End projects MLOps" from iNeuron.\n\nFrom\n\nFresher\n\nTo\n\n79% Increment\n\nSubham Kanungo\n\nAssociate Data Scientist\n\nI just joined EY as data analyst. It would not be possible with the support of Kri

### `Chunkings`

In [56]:
text_splitter = CharacterTextSplitter(separator = '\n',
                                      chunk_size = 1000,
                                      chunk_overlap = 200)

In [57]:
text_chunks = text_splitter.split_documents(data)
print(len(text_chunks))
text_chunks

6


[Document(page_content='Learning with iNeuron made \xa0<>\nTake your career to the next level with industry ready programs,\nAn entire learning ecosystem at your fingertips to make learning fun.\nChoose from a range of tech programs and make your next big career switch.\nExplore Courses\n55%\nAverage Salary Hike\n400+\nDifferent Courses\n10000+\nCareer Transitions\n400+\nHiring Partners\nLIVE NOW\nSupport System\nOur support system is live again, this time it is bigger, better and faster.\nExperience a tech community like never seen before\nTake me there\nOur Courses\nView all\nView all\nSuccess Stories\nView all\nFresher\nAbhisekh Bhuyan\nMLOps engineer\nI got job as an MLOps engineer at synapsica at 13 LPA PPO because of "End to End projects MLOps" from iNeuron.\nFrom\nFresher\nTo\n79% Increment\nSubham Kanungo\nAssociate Data Scientist\nI just joined EY as data analyst. It would not be possible with the support of Krish sir and Sudhanshu sir.\nFrom\nTo\n100% Increment\nSayan Saha\nS

In [58]:
text_chunks[2]

Document(page_content="From\nIntern\nTo\nFresher\nAdityaraj Chaudhari\nML Engineer\nI am delighted to inform you that I have successfully joined Zycus Infotech as trainee ML Engineer.\nFrom\nFresher\nTo\n300% Increment\nDisha Balpande\nML Engineer\nThank You so much for wonerful course and videos, I got a job at Adidas with 300% Hike.\nFrom\nTo\n150% Increment\nShubham Bharadwaj\nFinance Analyst\nI am beyond elated to announce that I have joined Deloitte USI as Analyst- Finance. Excited to move ahead in my carrer with biggest of the Big 4.\nFrom\nTo\n60% Increment\nPraveen Iyer\nSenior Software Engineer\nI have got an opportunity for a semiconductor chip manufacturing company in Bengaluru as Senior Software Engineer.\nFrom\nTo\nView all\nExplore Our Ecosystem\nA one-stop destination for all your learning to placement needs\nNeuro Lab\nGet premium access to a state-of-the-art virtual lab with infinite computing so you won't need additional investments in high-end PCs\nJob Portal", metad

### `Embedding Model`

In [59]:
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLm-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [60]:
query_result = embeddings.embed_query('Hello World')
len(query_result)

384

In [61]:
query_result

[-0.03447728976607323,
 0.03102319873869419,
 0.0067349751479923725,
 0.026109017431735992,
 -0.03936200216412544,
 -0.16030250489711761,
 0.06692400574684143,
 -0.006441437639296055,
 -0.047450508922338486,
 0.014758866280317307,
 0.07087535411119461,
 0.05552755296230316,
 0.019193312153220177,
 -0.02625133842229843,
 -0.010109523311257362,
 -0.026940515264868736,
 0.022307416424155235,
 -0.022226650267839432,
 -0.14969263970851898,
 -0.017493100836873055,
 0.007676243782043457,
 0.05435231328010559,
 0.003254458773881197,
 0.031725961714982986,
 -0.08462141454219818,
 -0.029405998066067696,
 0.05159568414092064,
 0.048124074935913086,
 -0.0033148014917969704,
 -0.05827920511364937,
 0.04196930304169655,
 0.022210700437426567,
 0.1281888335943222,
 -0.02233896777033806,
 -0.011656248942017555,
 0.06292837858200073,
 -0.03287629783153534,
 -0.09122606366872787,
 -0.031175322830677032,
 0.05269954353570938,
 0.04703483730554581,
 -0.08420302718877792,
 -0.030056210234761238,
 -0.020744

In [62]:
import os
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'ef065653-edca-4a9c-92c0-732b33d143b7')
PINECONE_API_HOST = os.environ.get('PINECONE_API_HOST', 'https://website-chatbot-xxbunxq.svc.aped-4627-b74a.pinecone.io') # Rotate and test this

In [63]:
#pc = Pinecone(api_key="YOUR_API_KEY")

In [66]:
import pinecone
# pinecone.init(
#     api_key = 'ef065653-edca-4a9c-92c0-732b33d143b7',
#     host = 'https://website-chatbot-xxbunxq.svc.aped-4627-b74a.pinecone.io')
pinecone.init(
    api_key = 'ef065653-edca-4a9c-92c0-732b33d143b7',
    environment = 'https://website-chatbot-xxbunxq.svc.aped-4627-b74a.pinecone.io'
    )
index_name = 'website-chatbot' # Mention Index name create in Pinecone

In [67]:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name = index_name)

ValueError: No active indexes found in your Pinecone project, are you sure you're using the right Pinecone API key and Environment? Please double check your Pinecone dashboard.

### `LLM Wrapper`

In [68]:
notebook_login()

In [69]:
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf", use_auth_token = True,)

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map = 'auto',
                                             torch_dtype = torch.float16,
                                             use_auth_token = True,
                                            #load_in_8bit = True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [76]:
pipeline = transformers.pipeline(
  #  model = model,
    model = 'meta-llama/Llama-2-7b-chat-hf',
    tokenizer = tokenizer,
    torch_dtype = torch.float16,
    device_map = 'auto',
    max_new_tokens = 512,
    do_sample = True,
    top_k = 30,
    num_return_sequences = 1,
    eos_token_id = tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [77]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})
# llm = HuggingFacePipeline(pipeline = pipeline)

In [78]:
llm.predict('Please provide a concise summary of the Book Harry Potter')

KeyboardInterrupt: 

### `Retrieval QA`

In [35]:
from langchain.chains import RetrievalQA

In [38]:
qa = RetrievalQA.from_chain_type(llm = llm, chain_type = 'stuff', retriever = docsearch.as_retriever())


In [ ]:
query = "Tell me about Cisco"

In [ ]:
print(qa.run(query))

## `Prompt Design`

In [4]:
!pip install langchain
!pip install bitsandbytes accelerate transformers -q
!pip install sentence_transformers
!pip install unstructured
!pip install -U langchain-community
!pip install pinecone-client==2.2.4

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
import torch, transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [8]:
model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token = True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [9]:
model = AutoModelForCausalLM.from_pretrained(model,
                                             device_map = 'auto',
                                             torch_dtype = torch.float16,
                                             use_auth_token = True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [13]:
pipeline = transformers.pipeline("text-generation",
                                 model = model,
                                 tokenizer = tokenizer,
                                 torch_dtype = torch.bfloat16,
                                 device_map = "auto",
                                 max_new_tokens = 512,
                                 min_new_tokens = -1,
                                 temperature = 0.75,
                                 do_sample = True,
                                 top_k = 30,
                                 num_return_sequences = 1,
                                 eos_token_id = tokenizer.eos_token_id)

- https://replicate.com/meta/llama-2-7b-chat

In [14]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature' :0})
llm.predict("Please provide a concise summary of the Book Harry Potter")

"Please provide a concise summary of the Book Harry Potter and the Philosopher's Stone.\nHarry Potter is a young boy who discovers that he is a wizard, despite being raised by his cruel and neglectful non-magical (Muggle) parents. On his eleventh birthday, Harry learns the truth about his parents and his own magical heritage from Rubeus Hagrid, the gamekeeper at Hogwarts School of Witchcraft and Wizardry. Harry begins attending Hogwarts and becomes friends with Ron Weasley and Hermione Granger, two other students who share his love for magic. Together, they embark on a journey to uncover the truth about the Philosopher's Stone, a powerful magical object that can grant eternal life. Along the way, they must confront the challenges of learning magic, navigating the complexities of friendship and loyalty, and facing the dark wizard Lord Voldemort, who murdered Harry's parents and seeks to return to power.\n\nPlease provide a summary of the book Harry Potter and the Philosopher's Stone by 

In [15]:
# Instruction Token, Instruction End (Beginning, Ending) This is the input prompt which we provide in the UI called as Instruction Prompt.
# System identifies the instruction prompt by the word "INST" and System prompt by the word "SYS"

B_INST, E_INST = "[INST]", "[/INST]" # Instruction Token, Instruction End (Beginning, Ending) This is the input prompt which we provide in the UI called as Instruction Prompt.
B_SYS, E_SYS = "<<SYS>>\n", "\n<<SYS>>\n\n" # This is the default prompt which will be present at the backend in the LLM

In [16]:
# Writing and Modifying the Default Prompt
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
"""

In [17]:
instruction = "Convert the following text from English to French: \n\n{text}"

In [18]:
SYSTEM_PROMPT = B_SYS + DEFAULT_SYSTEM_PROMPT + E_SYS
SYSTEM_PROMPT

"<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\n"

In [19]:
template = B_INST + SYSTEM_PROMPT + instruction +E_INST
template

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\nConvert the following text from English to French: \n\n{text}[/INST]"

### `Text Translation`

In [47]:
instruction = "Convert the following text from English to Hindi: \n\n{text}"

In [48]:
SYSTEM_PROMPT = B_SYS + DEFAULT_SYSTEM_PROMPT + E_SYS
SYSTEM_PROMPT

"<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\n"

In [49]:
template = B_INST + SYSTEM_PROMPT + instruction +E_INST
template

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\nConvert the following text from English to Hindi: \n\n{text}[/INST]"

### `Prompt Template`

In [50]:
prompt = PromptTemplate(input_variables = ["text"], template = template)

In [51]:
text = "How are you"

In [53]:
LLM_Chain = LLMChain(llm = llm, prompt = prompt)
print(LLM_Chain.run(text))

[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

<<SYS>>

Convert the following text from English to Hindi: 

How are you[/INST]  I apologize, but I cannot provide a direct conversion of the text "How are you" from English to Hindi as it is a common phrase used in English language and does not have an exact equivalent in Hindi. However, I can suggest a possible translation of the phrase "How are you?" in Hindi as "तुम्हारा स्वास्थ्य कैसा है" (Tumhārā swāsthya kaisā hai). Please note that the translation may vary dependin

### `Custom Prompt Template`
- `Custom Prompt Template` avoids the unnecessary words which will be generated by the model as the results.

In [40]:
instruction = "Convert the following text from English to French:\n\n{text}"

In [41]:
CUSTOM_SYSTEM_PROMPT = "You are an advanced assistant that excels at the translation."

In [42]:
SYSTEM_PROMPT = B_SYS + CUSTOM_SYSTEM_PROMPT + E_SYS
SYSTEM_PROMPT

'<<SYS>>\nYou are an advanced assistant that excels at the translation.\n<<SYS>>\n\n'

In [43]:
template = B_INST + SYSTEM_PROMPT + instruction +E_INST
template

'[INST]<<SYS>>\nYou are an advanced assistant that excels at the translation.\n<<SYS>>\n\nConvert the following text from English to French:\n\n{text}[/INST]'

In [44]:
prompt = PromptTemplate(input_variables = ["text"], template = template)

In [45]:
text = "How are you"

In [46]:
LLM_Chain = LLMChain(llm = llm, prompt = prompt)
print(LLM_Chain.run(text))

[INST]<<SYS>>
You are an advanced assistant that excels at the translation.
<<SYS>>

Convert the following text from English to French:

How are you[/INST]  Sure! Here is the translation of "How are you" from English to French:

Comment vas-tu?
